In [6]:
from RT_prediction.RT_prediction import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
#define predictor object, n_jobs sets the number of parralel processes to use
RT_pred = RT_predictor(n_jobs=4)

#get training data from pHILIC. This is molecular descriptors and RTs
training_data = RT_pred.get_training_data_pHILIC_IROA()

#get descriptors that can be used for training data
goodCols_train = RT_pred.get_good_desc(training_data)


Unnamed: 0 : non-real valued descriptor
Name : non-real valued descriptor


In [8]:
toPredict = pd.read_csv("../data/example_smiles.csv",index_col=0)
print(toPredict)
pred_props = RT_pred.computeMolecularDescriptors(toPredict)
pred_props = pred_props.set_index(pred_props[pred_props.columns.values[-1]])

pred_props

                                          SMILES
name                                            
cis-aconitic acid  C(/C(=C/C(=O)O)/C(=O)O)C(=O)O
citric acid         C(C(=O)O)C(CC(=O)O)(C(=O)O)O
P5C                         C1C[C@H](N=C1)C(=O)O


,Unnamed: 0,Wlambda1.unity,Wlambda2.unity,Wlambda3.unity,Wnu1.unity,Wnu2.unity,Wgamma1.unity,Wgamma2.unity,Wgamma3.unity,Weta1.unity,...,GRAV.1,GRAV.2,GRAV.3,GRAVH.1,GRAVH.2,GRAVH.3,GRAV.4,GRAV.5,GRAV.6,"data[i, 1]"
"data[i, 1]",,,,,,,,,,,,,,,,,,,,,
cis-aconitic acid,C(/C(=C/C(=O)O)/C(=O)O)C(=O)O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cis-aconitic acid
citric acid,C(C(=O)O)C(CC(=O)O)(C(=O)O)O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,citric acid
P5C,C1C[C@H](N=C1)C(=O)O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P5C


In [9]:


#these properties are computed by running the R script compute_molecular_properites.R. 
goodCols_pred = RT_pred.get_good_desc(pred_props)
preds = list(set(goodCols_train).intersection(set(goodCols_pred)))
print(len(preds),preds)

#train model and print cv r2
nBest,mBest,max_depth = (100,51,200)

if len(preds) < mBest:
    mBest = len(preds)
    
print(RT_pred.crossValPrediction(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth))
RT_pred.fit(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth)

Unnamed: 0 : non-real valued descriptor
data[i, 1] : non-real valued descriptor
86 ['MDEC.33', 'VCH.7', 'VP.3', 'SP.0', 'SCH.7', 'VAdjMat', 'ATSp5', 'WTPT.1', 'tpsaEfficiency', 'MDEC.22', 'ATSp3', 'nSmallRings', 'nHBAcc', 'nHBDon', 'nAtomP', 'WTPT.3', 'SP.4', 'SP.3', 'nRings5', 'WPOL', 'FMF', 'C2SP3', 'SPC.4', 'nRingBlocks', 'nAtomLC', 'nAcid', 'SP.5', 'nB', 'MDEC.34', 'VC.4', 'VP.5', 'VP.2', 'SP.2', 'VC.5', 'SC.5', 'SC.6', 'WTPT.5', 'ATSm3', 'AMR', 'MDEO.11', 'VPC.4', 'bpol', 'MDEC.24', 'apol', 'VP.0', 'ATSm5', 'nRotB', 'ATSp4', 'Zagreb', 'VPC.6', 'SP.1', 'SC.4', 'ALogp2', 'MDEC.23', 'C3SP2', 'ATSm4', 'MW', 'SPC.5', 'VP.1', 'ATSp1', 'VP.6', 'VP.4', 'VC.3', 'C3SP3', 'VC.6', 'C2SP2', 'ATSm2', 'SCH.5', 'C1SP2', 'SPC.6', 'fragC', 'XLogP', 'SCH.6', 'VCH.5', 'WPATH', 'ATSp2', 'SC.3', 'MLogP', 'ALogP', 'VPC.5', 'ATSm1', 'ECCEN', 'WTPT.4', 'TopoPSA', 'VCH.6', 'SP.6']
0.5461044660542731


In [10]:
#predict RTs
rt_predicted = RT_pred.predict(pred_props[preds].to_numpy())

#get cpd names
names = pred_props[pred_props.columns.values[-1]].values

#output RTs
pd.DataFrame.from_dict({n:{"name":n,"RT":rt} for n,rt in zip(names,rt_predicted)},orient="index").to_csv("../data/example_predicted_rts.csv",index=False)

In [11]:

#predict RTs
rt_predicted = RT_pred.predict(pred_props[preds].to_numpy())

#get cpd names
names = pred_props[pred_props.columns.values[-1]].values

#output RTs
pd.DataFrame.from_dict({n:{"name":n,"RT":rt} for n,rt in zip(names,rt_predicted)},orient="index").to_csv("../data/example_predicted_rts.csv",index=False)